In [15]:
from collections import defaultdict, Counter
import re

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BPE Tokenizer") \
    .getOrCreate()


24/05/20 20:05:33 WARN Utils: Your hostname, trnmah-IdeaPad-Gaming-3-15ACH6 resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/05/20 20:05:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 20:05:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
text_file = "./alice.txt"
text_rdd = spark.sparkContext.textFile(text_file)


In [9]:
text_rdd.take(1)

['Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?” So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and 

In [13]:
mapped_text = text_rdd.flatMap(lambda line: line.split(" ")).take(10)

In [14]:
mapped_text

['Alice',
 'was',
 'beginning',
 'to',
 'get',
 'very',
 'tired',
 'of',
 'sitting',
 'by']

In [19]:
# def build_vocab(rdd):
#     vocab = Counter()
#     for line in rdd.collect():
#         tokens = tokenize(line)
#         vocab[' '.join(tokens) + ' </w>'] += 1
#     return vocab
# build_vocab(mapped_text)

Counter({'Alice was beginning to get very tired of sitting by </w>': 10})

In [ ]:
# def build_vocab(rdd):
#     vocab = Counter()
#     for line in rdd.collect():
#         tokens = tokenize(line)
#         vocab[' '.join(tokens) + ' </w>'] += 1
#     return vocab

In [12]:
text_rdd.collect()

['Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?” So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and 

In [11]:
def get_pairs(word):
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs
get_pairs("hello")

{('e', 'l'), ('h', 'e'), ('l', 'l'), ('l', 'o')}

In [24]:
diction = {
    'a':2,
    'b':3
}
print(max(diction, key=diction.get))
print(max(diction,  ))

b
b


In [31]:
a = ('a@', 'b')
re.escape(''.join(a))

'a@b'

In [32]:
# Tokenize the text into characters
def tokenize(text):
    return text.split()

# Get BPE Pairs
def get_pairs(word):
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

# BPE Learn Function
def learn_bpe(vocab, num_merges):
    for i in range(num_merges):
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            for pair in get_pairs(word):
                pairs[pair] += freq
        if not pairs:
            break

        best = max(pairs, key=pairs.get)
        bigram = re.escape(' '.join(best))
        pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        
        vocab_out = {}
        for word in vocab:
            w_out = pattern.sub(''.join(best), word)
            vocab_out[w_out] = vocab[word]
        vocab = vocab_out

    return vocab

# Prepare Vocabulary
def build_vocab(rdd):
    vocab = Counter()
    for line in rdd.collect():
        tokens = tokenize(line)
        for token in tokens:
            vocab[token] += 1
        # vocab[' '.join(tokens) + ' </w>'] += 1
    return vocab


num_merges = 1000  # Define the number of merges
vocab = build_vocab(text_rdd)
# print(vocab)
vocab = learn_bpe(vocab, num_merges)

# Display the learned vocabulary
for word, freq in vocab.items():
    print(f"{word}: {freq}")


Alice: 4
was: 5
beginning: 1
to: 9
get: 1
very: 4
tired: 1
of: 8
sitting: 1
by: 2
her: 7
sister: 2
on: 1
the: 13
bank,: 1
and: 7
having: 1
nothing: 2
do:: 1
once: 1
or: 4
twice: 1
she: 7
had: 3
peeped: 1
into: 1
book: 1
reading,: 1
but: 3
it: 7
no: 1
pictures: 2
conversations: 1
in: 4
it,: 4
“and: 1
what: 1
is: 1
use: 1
a: 8
book,”: 1
thought: 2
“without: 1
conversations?”: 1
So: 1
considering: 1
own: 1
mind: 2
(as: 1
well: 1
as: 1
could,: 1
for: 2
hot: 1
day: 1
made: 1
feel: 1
sleepy: 1
stupid),: 1
whether: 1
pleasure: 1
making: 1
daisy-chain: 1
would: 1
be: 2
worth: 1
trouble: 1
getting: 1
up: 1
picking: 1
daisies,: 1
when: 2
suddenly: 1
White: 1
Rabbit: 3
with: 3
pink: 1
eyes: 1
ran: 2
close: 1
her.: 1
There: 1
so: 2
remarkable: 1
that;: 1
nor: 1
did: 1
think: 1
much: 1
out: 3
way: 1
hear: 1
say: 1
itself,: 1
“Oh: 1
dear!: 2
Oh: 1
I: 1
shall: 1
late!”: 1
(when: 1
over: 1
afterwards,: 1
occurred: 1
that: 2
ought: 1
have: 1
wondered: 1
at: 3
this,: 1
time: 2
all: 1
seemed: 1
quite: 1


In [33]:
import json

vocab_file = "./output.txt"
with open(vocab_file, 'w') as f:
    json.dump(vocab, f)
